# CPSC 330 - Applied Machine Learning 

## Homework 6: Putting it all together 
### Associated lectures: All material till lecture 13 

**Due date: Wednesday, March 15, 2023 at 11:59pm**

## Table of contents

- [Submission instructions](#si)
- [Understanding the problem](#1)
- [Data splitting](#2)
- [EDA](#3)
- (Optional) [Feature engineering](#4)
- [Preprocessing and transformations](#5)
- [Baseline model](#6)
- [Linear models](#7)
- [Different classifiers](#8)
- (Optional) [Feature selection](#9)
- [Hyperparameter optimization](#10)
- [Interpretation and feature importances](#11)
- [Results on the test set](#12)
- (Optional) [Explaining predictions](#13)
- [Summary of the results](#14)

## Imports 

In [ ]:
import os

%matplotlib inline
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    make_scorer,
    plot_confusion_matrix,
)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC

import pandas_profiling as pp

<br><br>

## Instructions 
<hr>
rubric={points:2}

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2022W2/blob/main/docs/homework_instructions.md). 

**You may work on this homework in a group and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 3. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).

<br><br>

## Introduction <a name="in"></a>
<hr>

At this point we are at the end of supervised machine learning part of the course. So in this homework, you will be working on an open-ended mini-project, where you will put all the different things you have learned so far together to solve an interesting problem.

A few notes and tips when you work on this mini-project: 

#### Tips

1. This mini-project is open-ended, and while working on it, there might be some situations where you'll have to use your own judgment and make your own decisions (as you would be doing when you work as a data scientist). **Make sure you explain your decisions whenever necessary.** 
2. **Do not include everything you ever tried in your submission** -- it's fine just to have your final code. That said, your code should be reproducible and well-documented. For example, if you chose your hyperparameters based on some hyperparameter optimization experiment, you should leave in the code for that experiment so that someone else could re-run it and obtain the same hyperparameters, rather than mysteriously just setting the hyperparameters to some (carefully chosen) values in your code. 
3. If you realize that you are repeating a lot of code try to organize it in functions. Clear presentation of your code, experiments, and results is the key to be successful in this lab. You may use code from lecture notes or previous lab solutions with appropriate attributions. 
4. If you are having trouble running models on your laptop because of the size of the dataset, you can create your train/test split in such a way that you have less data in the train split. If you end up doing this, please write a note to the grader in the submission explaining why you are doing it.  

#### Assessment

We plan to grade fairly and leniently. We don't have some secret target score that you need to achieve to get a good grade. **You'll be assessed on demonstration of mastery of course topics, clear presentation, and the quality of your analysis and results.** For example, if you just have a bunch of code and no text or figures, that's not good. If you do a bunch of sane things and get a lower accuracy than your friend, don't sweat it.

#### A final note

Finally, this style of this "project" question is different from other assignments. It'll be up to you to decide when you're "done" -- in fact, this is one of the hardest parts of real projects. But please don't spend WAY too much time on this... perhaps "a few hours" (2-8 hours???) is a good guideline for a typical submission. Of course if you're having fun you're welcome to spend as much time as you want! But, if so, try not to do it out of perfectionism or getting the best possible grade. Do it because you're learning and enjoying it. Students from the past cohorts have found such kind of labs useful and fun and I hope you enjoy it as well. 

<br><br>

## 1. Understanding the problem <a name="1"></a>
<hr>
rubric={points:4}

In this mini project, you will be working on a classification problem of predicting whether a customer will cancel the reservation they have made at a hotel. 
For this problem, you will use [Reservation Cancellation Prediction Dataset](https://www.kaggle.com/datasets/gauravduttakiit/reservation-cancellation-prediction?select=train__dataset.csv). In this data set, there are about 18.000 examples and 18 features (including the target), and the goal is to estimate whether a person will cancel their booking; this column is labeled "booking_status" in the data (1 = canceled). 

**Your tasks:**

1. Spend some time understanding the problem and what each feature means. You can find this information in the documentation on [the dataset page on Kaggle](https://www.kaggle.com/datasets/gauravduttakiit/reservation-cancellation-prediction?select=train__dataset.csv). Write a few sentences on your initial thoughts on the problem and the dataset. 
2. Download the dataset and read it as a pandas dataframe. 

Looking at the features, the data types all seem to be numbers but not all are numerical features. I'm seeing some potential categorical or binary features which I'll expand on in another part. I'm not sure how relevant the market_segment_type feature will be, but the features that record the number of previous cancellations and non-cancellations feel like they might be important.

In [ ]:
df = pd.read_csv("train__dataset.csv", encoding="utf-8")
df

<br><br>

## 2. Data splitting <a name="2"></a>
<hr>
rubric={points:2}

**Your tasks:**

1. Split the data into train and test portions. 

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=123)
X_train = train_df.drop(columns=["booking_status"])
X_test = test_df.drop(columns=["booking_status"])

y_train = train_df["booking_status"]
y_test = test_df["booking_status"]

<br><br>

## 3. EDA <a name="3"></a>
<hr>
rubric={points:10}

**Your tasks:**

1. Perform exploratory data analysis on the train set.
2. Include at least two summary statistics and two visualizations that you find useful, and accompany each one with a sentence explaining it.
3. Summarize your initial observations about the data. 
4. Pick appropriate metric/metrics for assessment. 

In [ ]:
display(train_df.info())
display(train_df.describe())

I can see in the first statistic that we don't have missing values and that the data types are all integers or floats. In the second statistic that I'll require standard scaling to the numeric features. Also, some of the features like repeated_guest and required_car_parking_space might be binary.

pr = pp.ProfileReport(train_df)
pr.to_file("profilereport.html") #exporting as html to save space when submitting this notebook

In [ ]:
booking_kept = train_df.query("booking_status == 1")
booking_cancelled = train_df.query("booking_status == 0")
plt.hist(x=booking_kept["lead_time"], label="kept", alpha=0.5, bins=20)
plt.hist(x=booking_cancelled["lead_time"], label="cancelled", alpha=0.5, bins=20)
plt.xlabel("lead_time")
plt.ylabel("count")
plt.legend(loc="upper left")
plt.title(f"Histogram of booking cancellations by lead time")
plt.show()

Canceled bookings seem to skew towards the lower end of lead times. This may indicate that this feature is important.

In [ ]:
train_df.groupby("booking_status").size().plot.bar(title="Bookings Kept and Cancelled", xlabel="booking_status", ylabel="Count", rot=0)

This bar chart shows that there's a lot more non-canceled bookings than there are cancellations. The imbalance isn't as extreme as previous example, but still, accuracy won't be a good metric alone. Since we want to predict whether a customer will cancel or not, the f1-score would be a good metric since precision and recall are important as well. The project description didn't mention any real-world application of this, so I'm not going to focus too much on recall or precision over the other. (This would be different if I had to consider the cost of FNs or FPs).

<br><br>

## (Optional) 4. Feature engineering <a name="4"></a>
<hr>
rubric={points:1}

**Your tasks:**

1. Carry out feature engineering. In other words, extract new features relevant for the problem and work with your new feature set in the following exercises. You may have to go back and forth between feature engineering and preprocessing. 

<br><br>

## 5. Preprocessing and transformations <a name="5"></a>
<hr>
rubric={points:10}

**Your tasks:**

1. Identify different feature types and the transformations you would apply on each feature type. 
2. Define a column transformer, if necessary. 

In [ ]:
# I'll identify the different feature types and transformations here. There is no missing data according to the EDA, so I'll skip imputation
numeric_features = [
    "no_of_adults",
    "no_of_children",
    "no_of_weekend_nights",
    "no_of_week_nights",
    "lead_time",
    "no_of_previous_cancellations",
    "no_of_previous_bookings_not_canceled",
    "avg_price_per_room",
    "no_of_special_requests",
]

#I considered ordinal for 
categorical_features = ["type_of_meal_plan", "room_type_reserved", "market_segment_type", "arrival_year", "arrival_month", "arrival_date"]
binary_features = ["required_car_parking_space", "repeated_guest"]

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore", sparse=False)
binary_transformer = OneHotEncoder(drop="if_binary", dtype=int)

column_transformer = make_column_transformer(
    (numeric_transformer, numeric_features),
    (categorical_transformer, categorical_features),
    (binary_transformer, binary_features)
)

transformed = column_transformer.fit_transform(X_train, y_train)
transformed

<br><br>

X_train## 6. Baseline model <a name="6"></a>
<hr>

rubric={points:2}

**Your tasks:**
1. Try `scikit-learn`'s baseline model and report results.

In [ ]:
scoring = [
    "accuracy",
    "f1",
    "recall",
    "precision",
]

pipeline = make_pipeline(column_transformer, DummyClassifier(strategy="prior"))
scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, scoring=scoring)
pd.DataFrame(scores)

Getting some warnings since the DummyClassifier is always predicting 0, so 0 division is happening when precision is being computed.

<br><br>

## 7. Linear models <a name="7"></a>
<hr>
rubric={points:12}

**Your tasks:**

1. Try logistic regression as a first real attempt. 
2. Carry out hyperparameter tuning to explore different values for the complexity hyperparameter `C`. 
3. Report validation scores along with standard deviation. 
4. Summarize your results.

In [ ]:
lr = LogisticRegression(max_iter=1000, random_state=123)
pipeline = make_pipeline(column_transformer, lr)

scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, scoring=scoring)
pd.DataFrame(scores)

In [ ]:
scores = {
    "C": C_vals, 
    "train_accuracy": [], 
    "test_accuracy": [], 
    "test_accuracy_std": [],
    "test_f1": [],
    "test_recall": [],
    "test_precision": [],
}

C_vals = 10.0 ** np.arange(-1.5, 2.5, 0.5)

for C in C_vals:
    pipeline = make_pipeline(column_transformer, LogisticRegression(max_iter=1000, random_state=123, C=C))    
    cv_results = cross_validate(pipeline, X_train, y_train, return_train_score=True, scoring=scoring)

    scores["train_accuracy"].append(cv_results["train_accuracy"].mean())
    scores["test_accuracy"].append(cv_results["test_accuracy"].mean())
    scores["test_accuracy_std"].append(cv_results["test_accuracy"].std())
    scores["test_f1"].append(cv_results["test_f1"].mean())
    scores["test_recall"].append(cv_results["test_recall"].mean())
    scores["test_precision"].append(cv_results["test_precision"].mean())

pd.DataFrame(scores)

In [ ]:
pipeline = make_pipeline(column_transformer, LogisticRegression(max_iter=1000, random_state=123, C=0.316228))
scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, scoring=scoring)
pd.DataFrame(scores)

Changing C doesn't seem to make a huge difference beyond 0.316228 in nearly all metrics apart from recall.
Referenced code from HW 4.

<br><br>

## 8. Different classifiers <a name="8"></a>
<hr>
rubric={points:15}

**Your tasks:**
1. Try at least 3 other models aside from logistic regression. At least one of these models should be a tree-based ensemble model (e.g., lgbm, random forest, xgboost). 
2. Summarize your results. Can you beat logistic regression? 

In [ ]:
scores_summary = {
    "LogisticRegression": pd.DataFrame(scores).mean()
}

In [ ]:
pipeline = make_pipeline(column_transformer, SVC(random_state=123))
scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, scoring=scoring)
scores_summary["SVC"] = pd.DataFrame(scores).mean()
pd.DataFrame(scores)

In [ ]:
pipeline = make_pipeline(column_transformer, RandomForestClassifier(random_state=123, n_jobs=-1))
scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, scoring=scoring)
scores_summary["RandomForestClassifier"] = pd.DataFrame(scores).mean()
pd.DataFrame(scores)

In [ ]:
pipeline = make_pipeline(column_transformer, xgb.XGBClassifier(random_state=123, eval_metric="logloss", verbosity=0))
scores = cross_validate(pipeline, X_train, y_train, return_train_score=True, scoring=scoring)
scores_summary["XGBClassifier"] = pd.DataFrame(scores).mean()
pd.DataFrame(scores)

In [ ]:
pd.DataFrame(scores_summary).T

It looks like RandomForestClassifier beats out the other models in all metrics aside from recall (but it's very, very close with XGBClassifier). LogisticRegression is easily beaten by the others.
Referenced lecture 11.

<br><br>

## (Optional) 9. Feature selection <a name="9"></a>
<hr>
rubric={points:1}

**Your tasks:**

Make some attempts to select relevant features. You may try `RFECV` or forward selection. Do the results improve with feature selection? Summarize your results. If you see improvements in the results, keep feature selection in your pipeline. If not, you may abandon it in the next exercises. 

<br><br>

## 10. Hyperparameter optimization <a name="10"></a>
<hr>
rubric={points:15}

**Your tasks:**

Make some attempts to optimize hyperparameters for the models you've tried and summarize your results. You may pick one of the best performing models from the previous exercise and tune hyperparameters only for that model. You may use `sklearn`'s methods for hyperparameter optimization or fancier Bayesian optimization methods. 
  - [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)   
  - [RandomizedSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html)
  - [scikit-optimize](https://github.com/scikit-optimize/scikit-optimize)

In [ ]:
pipeline = make_pipeline(column_transformer, RandomForestClassifier(random_state=123, n_jobs=-1))

n_estimators = [100, 300, 500, 700, 900]
max_features = ["sqrt", "log2", None]

param_grid = {
    "randomforestclassifier__n_estimators": n_estimators,
    "randomforestclassifier__max_features": max_features,
}
#commenting out to save time on final run
#grid_search = GridSearchCV(pipeline, param_grid, scoring="f1", n_jobs=-1)
#grid_search.fit(X_train, y_train)
#best_n_estimators = grid_search.best_params_['randomforestclassifier__n_estimators']
#best_max_features = grid_search.best_params_['randomforestclassifier__max_features']
#best_score = grid_search.best_score_

print(f"best_n_estimators: {best_n_estimators}\nbest_max_features:{best_max_features}\nbest_score:{best_score}")

best_n_estimators: 700

best_max_features:None 

best_score:0.8165093261552311 

Referenced https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

<br><br>

## 11. Interpretation and feature importances <a name="1"></a>
<hr>
rubric={points:15}

**Your tasks:**

1. Use the methods we saw in class (e.g., `eli5`, `shap`) (or any other methods of your choice) to explain feature importances of one of the best performing models. Summarize your observations. 

<br><br>

## 12. Results on the test set <a name="12"></a>
<hr>

rubric={points:5}

**Your tasks:**

1. Try your best performing model on the test data and report test scores. 
2. Do the test scores agree with the validation scores from before? To what extent do you trust your results? Do you think you've had issues with optimization bias? 

In [ ]:
pipeline = make_pipeline(column_transformer, RandomForestClassifier(random_state=123, n_estimators=700, max_features=None, n_jobs=-1))
pipeline.fit(X_train, y_train)
predictions = pipeline.predict(X_test)
score = f1_score(y_test, predictions)
score

<br><br>

## (Optional) 13. Explaining predictions 
rubric={points:1}

**Your tasks**

1. Take one or two test predictions and explain them with SHAP force plots.  

<br><br>

## 14. Summary of results <a name="13"></a>
<hr>
rubric={points:10}

**Your tasks:**

1. Report your final test score along with the metric you used. 
2. Write concluding remarks.
3. Discuss other ideas that you did not try but could potentially improve the performance/interpretability . 

Ran out of time :(

<br><br><br><br>

## Submission instructions 

**PLEASE READ:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission. 